In [1]:
import numpy as np
import pandas as pd
import numpy.typing as npt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import os

# Load imputed data
data = pd.read_csv("../0_DataPreparation/processed_data_imputed.csv")

# Load deleted data
#data = pd.read_csv("../0_DataPreparation/processed_data_deleted.csv")

# Filter out rows with Umsatz = 0
data = data[data['Umsatz'] != 0]

# Ensure 'Datum' is in datetime format
data['Datum'] = pd.to_datetime(data['Datum'], errors='coerce')

# Define feature columns
feature_columns = [
    'KiWo', 'Is_Weekend', 'Temperature_Category',
    'Wind_Status', 'Rain_Status',
    'Cloud_Status','Is_Ferien','Sommerferien_Flag', 'Christmas_Sales','Warengruppe_1', 'Warengruppe_2',
    'Warengruppe_3', 'Warengruppe_4', 'Warengruppe_5', 'Warengruppe_6'
]

print("Final feature columns used in training:", feature_columns)

# Split data into training and validation sets
training_start_date = '2013-07-01'
training_end_date = '2017-07-31'
validation_start_date = '2017-08-01'
validation_end_date = '2018-07-31'

train_data = data[(data['Datum'] >= training_start_date) & (data['Datum'] <= training_end_date)]
val_data = data[(data['Datum'] >= validation_start_date) & (data['Datum'] <= validation_end_date)]

X_train = train_data[feature_columns].to_numpy()
y_train = train_data['Umsatz'].to_numpy()
X_val = val_data[feature_columns].to_numpy()
y_val = val_data['Umsatz'].to_numpy()

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Build the neural network model
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),  # Input layer
    Dense(32, activation='relu'),                             # Hidden layer
    Dense(16, activation='relu'),                             # Hidden layer
    Dense(1, activation='linear')                             # Output layer for regression
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    verbose=1
)

# Evaluate the model
y_val_pred = model.predict(X_val).flatten()
mse = mean_squared_error(y_val, y_val_pred)
mape = mean_absolute_percentage_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)
validation_cost = mse * len(y_val) / 2

print(f"Best R²: {r2:.4f}")
print(f"Validation cost: {validation_cost:.4e}")
print(f"Mean Squared Error (MSE) on validation set: {mse:.4e}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

# Extract and display model parameters (weights from the first layer)
weights, biases = model.layers[0].get_weights()

print("\nFinal Model Parameters:")
for i, feature in enumerate(feature_columns):
    print(f"Feature: {feature}, Weight: {weights[i][0]:.4f}")
print(f"Intercept (b): {biases[0]:.4f}")

# Save the trained model
model_save_path = "/workspaces/Team_Raum-3_BakerySalesPredictions/3_Model/nn_model.h5"
os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
model.save(model_save_path)
print(f"Model saved to: {model_save_path}")


2025-01-09 12:01:04.910858: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-09 12:01:04.924209: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-09 12:01:05.144280: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-09 12:01:05.295779: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736424065.592182    4479 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736424065.69

Final feature columns used in training: ['KiWo', 'Is_Weekend', 'Temperature_Category', 'Wind_Status', 'Rain_Status', 'Cloud_Status', 'Is_Ferien', 'Sommerferien_Flag', 'Christmas_Sales', 'Warengruppe_1', 'Warengruppe_2', 'Warengruppe_3', 'Warengruppe_4', 'Warengruppe_5', 'Warengruppe_6']
Epoch 1/100


/usr/local/python/3.12.1/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-01-09 12:01:09.468637: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


234/234 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 50211.3984 - mae: 172.3850 - val_loss: 4699.3770 - val_mae: 48.3467
Epoch 2/100
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5726.4272 - mae: 47.8052 - val_loss: 4142.8940 - val_mae: 44.5706
Epoch 3/100
234/234 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4314.9546 - mae: 43.5091 - val_loss: 4695.4453 - val_mae: 47.4123
Epoch 4/100
234/234 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 4845.7183 - mae: 43.7325 - val_loss: 3856.2004 - val_mae: 42.2479
Epoch 5/100
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5927.1616 - mae: 42.7856 - val_loss: 4085.1885 - val_mae: 43.3720
Epoch 6/100
234/234 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4481.1909 - mae: 40.3377 - val_loss: 3873.9614 - val_mae: 41.6170
Epoch 7/100
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3305.9006 - mae: 39.0635 - val_loss: 4134.9727 - val_mae: 43.6115
Epoch 8/100
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3626.9421 - mae: 38.9576 - val_loss: 3743.8101 - val_mae: 40

Best R²: 0.7874
Validation cost: 3.3012e+06
Mean Squared Error (MSE) on validation set: 3.6059e+03
Mean Absolute Percentage Error (MAPE): 0.24%

Final Model Parameters:
Feature: KiWo, Weight: -0.5307
Feature: Is_Weekend, Weight: 0.3198
Feature: Temperature_Category, Weight: 0.2375
Feature: Wind_Status, Weight: -0.9970
Feature: Rain_Status, Weight: 0.0878
Feature: Cloud_Status, Weight: -0.2658
Feature: Is_Ferien, Weight: 0.2508
Feature: Sommerferien_Flag, Weight: 0.2536
Feature: Christmas_Sales, Weight: 0.2909
Feature: Warengruppe_1, Weight: -0.6942
Feature: Warengruppe_2, Weight: 0.2326
Feature: Warengruppe_3, Weight: -0.1589
Feature: Warengruppe_4, Weight: -0.5639
Feature: Warengruppe_5, Weight: 0.3759
Feature: Warengruppe_6, Weight: -0.1842
Intercept (b): -0.1304
Model saved to: /workspaces/Team_Raum-3_BakerySalesPredictions/3_Model/nn_model.h5


In [2]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
from sklearn.preprocessing import StandardScaler

# Paths to the files
processed_data_path = "/workspaces/Team_Raum-3_BakerySalesPredictions/0_DataPreparation/processed_data_imputed.csv"
sample_submission_path = "/workspaces/Team_Raum-3_BakerySalesPredictions/0_DataPreparation/sample_submission.csv"
final_submission_path = "/workspaces/Team_Raum-3_BakerySalesPredictions/3_Model/final_submission.csv"
model_path = "/workspaces/Team_Raum-3_BakerySalesPredictions/3_Model/nn_model.h5"

# Load the processed data and sample submission
processed_data = pd.read_csv(processed_data_path)
sample_submission = pd.read_csv(sample_submission_path)

# Dynamically rebuild feature columns
feature_columns = [
    'KiWo', 'Is_Weekend', 'Temperature_Category',
    'Wind_Status', 'Rain_Status',
    'Cloud_Status','Is_Ferien','Sommerferien_Flag', 'Christmas_Sales','Warengruppe_1', 'Warengruppe_2',
    'Warengruppe_3', 'Warengruppe_4', 'Warengruppe_5', 'Warengruppe_6'
]

# Extract features for prediction
X_new = processed_data[feature_columns].apply(pd.to_numeric, errors='coerce').fillna(0).to_numpy(dtype=np.float64)

# Load the trained neural network model
model = load_model(
    model_path,
    custom_objects={'mse': MeanSquaredError()}  # Ensure compatibility with saved model
)

# Normalize features using the same scaler used during training
scaler = StandardScaler()
X_new = scaler.fit_transform(X_new)  # Use saved scaler if available

# Predict the output using the trained neural network\n",
y_pred = model.predict(X_new).flatten()

# Add the predicted values (y_pred) to the processed data
processed_data['Predicted_Umsatz'] = y_pred

# Merge predictions with sample submission to ensure matching structure
final_submission = sample_submission.merge(processed_data[['id', 'Predicted_Umsatz']], on='id', how='left')

# Replace the existing 'Umsatz' column in sample_submission with 'Predicted_Umsatz'
final_submission['Umsatz'] = final_submission['Predicted_Umsatz']

# Drop the 'Predicted_Umsatz' column after replacement
final_submission.drop(columns=['Predicted_Umsatz'], inplace=True)

# Replace null values in the Umsatz column with 0
final_submission['Umsatz'] = final_submission['Umsatz'].fillna(0)

# Round the 'Umsatz' values to 2 decimal places
final_submission['Umsatz'] = final_submission['Umsatz'].round(2)

# Keep only 'id' and 'Umsatz' columns
final_submission = final_submission[['id', 'Umsatz']]

# Save the final submission file
os.makedirs(os.path.dirname(final_submission_path), exist_ok=True)
final_submission.to_csv(final_submission_path, index=False)

print(f"Final submission saved to: {final_submission_path}")




439/439 ━━━━━━━━━━━━━━━━━━━━ 0s 923us/step
Final submission saved to: /workspaces/Team_Raum-3_BakerySalesPredictions/3_Model/final_submission.csv
